In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 


In [2]:
data = pd.read_csv("../data/Teams.csv")

In [3]:
data["yearID"] = pd.to_datetime(data["yearID"],format="%Y").dt.year

In [4]:
year = 2000
data_s = data.loc[data["yearID"]>year]

In [12]:
data_s.drop(['lgID','teamID','franchID','divID','teamIDBR','teamIDlahman45','teamIDretro'],axis=1,inplace=True)

In [19]:
 team = list(data_s["name"].unique())

In [20]:

TEAMS = [
    "Chicago White Sox",
    "Cleveland Indians",
    "Detroit Tigers",
    "Kansas City Royals",
    "Minnesota Twins",
    "Baltimore Orioles",
    "Boston Red Sox",
    "New York Yankees",
    "Tampa Bay Rays",
    "Toronto Blue Jays",
    "Houston Astros",
    "Los Angeles Angels",
    "Oakland Athletics",
    "Seattle Mariners",
    "Texas Rangers",
    "Chicago Cubs",
    "Cincinnati Reds",
    "Milwaukee Brewers",
    "Pittsburgh Pirates",
    "St. Louis Cardinals",
    "Atlanta Braves",
    "Miami Marlins",
    "New York Mets",
    "Philadelphia Phillies",
    "Washington Nationals",
    "Arizona Diamondbacks",
    "Colorado Rockies",
    "Los Angeles Dodgers",
    "San Diego Padres",
    "San Francisco Giants"
]

In [26]:
for t in team:
    print(t)
    if t not in TEAMS:
        print("NOt in the list")


Anaheim Angels
NOt in the list
Arizona Diamondbacks
Atlanta Braves
Baltimore Orioles
Boston Red Sox
Chicago White Sox
Chicago Cubs
Cincinnati Reds
Cleveland Indians
Colorado Rockies
Detroit Tigers
Florida Marlins
NOt in the list
Houston Astros
Kansas City Royals
Los Angeles Dodgers
Milwaukee Brewers
Minnesota Twins
Montreal Expos
NOt in the list
New York Yankees
New York Mets
Oakland Athletics
Philadelphia Phillies
Pittsburgh Pirates
San Diego Padres
Seattle Mariners
San Francisco Giants
St. Louis Cardinals
Tampa Bay Devil Rays
NOt in the list
Texas Rangers
Toronto Blue Jays
Los Angeles Angels of Anaheim
NOt in the list
Washington Nationals
Tampa Bay Rays
Miami Marlins


In [24]:
len(TEAMS)

30